# Eksploracja Danych - Projekt
Tomasz Kiljańczyk (136257)

Wojciech Lulek (136280)

In [ ]:
import pandas as pd

In [ ]:
us_trending_df = pd.read_csv('./data/stage_1_us_trending.csv')
us_trending_df

In [ ]:
us_trending_df['thumbnail_link_hires'] = us_trending_df['thumbnail_link'].str.replace('default.jpg', '0.jpg')
urls = us_trending_df['thumbnail_link_hires']
unique_urls = us_trending_df['thumbnail_link_hires'].unique()
us_trending_df['thumbnail_link_hires']

# Using ImageAI for object detection

Trained model file is required! Put this into notebooks/models directory. Download link: https://github.com/OlafenwaMoses/ImageAI/releases/download/essentials-v5/resnet50_coco_best_v2.1.0.h5/

Checking gpu availability

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

Loading pre-trained model

In [ ]:
from imageai.Detection import ObjectDetection

detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath("models/resnet50_coco_best_v2.1.0.h5")
detector.loadModel()

Detecting objects from thumbnails

In [ ]:
from workers import file_path_from_url
import pickle
import tqdm

image_detections = dict()

for i, url in enumerate(tqdm.tqdm(unique_urls)):
    filename = file_path_from_url(url)
    detections = detector.detectObjectsFromImage(input_image=filename, output_type="array")[1]
    image_detections[url] = detections

    if i % 100 == 0:
        with open("data/thumbnails_objects_dict.pkl", "wb") as f:
            pickle.dump(image_detections, f)
with open("data/thumbnails_objects_dict.pkl", "wb") as f:
    pickle.dump(image_detections, f)

Loading detections from pickle file

In [ ]:
import pickle
import os

with open("data/thumbnails_objects_dict.pkl", "rb") as f:
    image_detections = dict(pickle.load(f))

os.remove("data/thumbnails_objects_dict.pkl")

Counting detected objects

In [ ]:
from collections import defaultdict, Counter

image_objects_detected = defaultdict(dict)
cnt = Counter()

for key in image_detections:
    for obj in image_detections[key]:
        name = obj['name']
        image_objects_detected[key][name] = image_objects_detected[key].get(name, 0) + 1
        cnt.update({name: 1})

print("counts", cnt)
print()
print("10 most common objects:", cnt.most_common(10))

Creating vector representation and saving it to data/ in CSV format

In [ ]:
import pandas as pd

image_objects_detected_full = [image_objects_detected.get(url, {}) for url in urls]

df = pd.DataFrame(image_objects_detected_full)
df = df.fillna(0)
df = df.astype('int32')

df.to_csv("data/stage_2_thumbnail_objects_vectors.csv", index=False)
df.head()

Notatka: Może wagi wilkością bboxów

może rozpoznawanie emocji na twarzach